# DS4A / COLOMBIA 4.0 

# Tema: Comportamiento del consumidor en el comercio electrónico

Data science for the digital future

- Ximena Rios Cotazo

- Ramon Manuel Sandoval 



## E-Commerce 

El comercio electrónico se define como la actividad de compra o venta de bienes a través de medios electrónicos en línea o en la red mundial. La llegada de Internet permitió un crecimiento sin precedentes de los canales de comunicación en todo el mundo. Como resultado, surgieron industrias y se desarrollaron mercados para compras en línea para satisfacer la demanda de productos que no estaban disponibles en el país.

El comercio electrónico ha dado lugar a un cambio en las estrategias de marketing y publicidad, lo que permite a las empresas llegar a los usuarios a través de múltiples canales (es decir, aplicaciones móviles) y expandir su alcance con un esfuerzo mínimo. En consecuencia, esto también ha generado cantidades sustanciales de datos del consumidor, que a través de un análisis matemático cuidadoso ofrece una capacidad poderosa pero perspicaz para reducir los comportamientos de gasto del consumidor, que a su vez impulsa sus ventas.

Además, la publicidad programática adaptada específicamente y la extracción de datos del consumidor ha provocado una nueva ola de empresas e industrias. El crecimiento exponencial de los datos generados a través de la "experiencia de compra en línea" permite a las empresas aprender y extraer información fundamental de las transacciones para desarrollar una visión de los segmentos de clientes, los sesgos, los precios y los hábitos de gasto.

## Planteamiento del problema

¿Qué aspectos del comportamiento en línea de los consumidores son útiles para que las empresas comprendan mejor a sus clientes y predigan las tendencias del consumidor, los hábitos de gasto y las variables relacionadas con las compras?

## Objetivos

- Comprensión y limpieza de datos
- Análisis y modelado
- Modelos de predicción
- Visualizacion de la información

## Requerimientos

In [1]:
import os
#import json
#import requests
#from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#import numpy

# DATASETS

Los conjuntos de datos proporcionados se distribuyen en cinco conjuntos primarios.

## DATASET: UK_retail_transactions 

Este conjunto de datos de Online tomado de [Retail](http://archive.ics.uci.edu/ml/datasets/Online+Retail+II). contiene todas las transacciones que se producen para una tienda en línea, con sede en el Reino Unido, entre el 01/12/2009 y el 09/12/2011. La empresa vende principalmente artículos de regalo únicos para todas las ocasiones. Muchos clientes de la empresa son mayoristas.

Las columnas presentadas en este archivo son:

|CAMPO   |TIPO   |DESCRIPCION  |    |
|---|---|---|---|
|invoice_no |STRING |Número de factura. |Un número entero de 6 dígitos asignado de forma exclusiva a cada transacción*.| 
|stock_code |STRING |Código de producto. |Un número entero de 5 dígitos asignado de forma única a cada producto.|
|description |STRING |Descripcion |Nombre del producto.|
|quantity |INT |Cantidad |Las cantidades de cada producto por transacción.|
|invoice_date |STRING |Fecha y hora |El día y la hora en que se realizó una transacción generado (formato: mm-dd-aa HH: MM). |
|unit_price |FLOAT |Precio |Precio del producto por unidad en libras esterlinas (libras).|
|customerid |FLOAT |Número de cliente. |Un número entero de 5 dígitos asignado de forma exclusiva a cada cliente.|
|country |STRING |Nombre del país. |El país donde reside un cliente.|

*Si este código comienza con la letra 'c', indica una cancelación.

In [2]:
ruta=os.getcwd()

In [3]:
#Cargamos el dataset que vamos a analizar.
df_UK=pd.read_csv(ruta+"/data/UK_retail_transactions.csv")

In [4]:
df_UK.shape #Tamaño del DataSet

(1067371, 8)

Este dataset contiene 1'067.371 transaciones, con 8 atributos (columnas)

In [5]:
df_UK.head(5) #Primeros registros

,invoice_no,stock_code,description,quantity,invoice_date,price,customerid,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/09 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/09 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/09 7:45,1.25,13085.0,United Kingdom


## Comprensión y limpieza de datos

### Eliminacion de elementos nulos

In [6]:
df_UK.isnull().sum() #Cantidad de Nulos por columna

invoice_no           0
stock_code           0
description       4382
quantity             0
invoice_date         0
price                0
customerid      243007
country              0
dtype: int64

El dataset contine 4.382 valores nulos en ```description``` y 243.007 valores nulos que corresponden al numero de cliente (```customerid```). Debido a que el ```custumerid``` es un dato fundamental en el analisis de datos que realizaremos, decidimos eliminar las filas que corresponden a esos registros.

In [7]:
#df_UK[(df_UK["customerid"].isnull())] #registros con customerid nulo

In [8]:
df_UK=df_UK[(df_UK["customerid"].isnull()==False)]  #borrar los nulos

In [9]:
df_UK.isnull().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
price           0
customerid      0
country         0
dtype: int64

In [10]:
df_UK.shape #Tamaño del DataSet

(824364, 8)

Ahora tenemos un dataset con 824.364 registros de transaciones que no contienen nulos

### Distribuciones marginales 

Es bueno desarrollar primero una idea de cómo se distribuyen los valores de algunas cantidades de interés. Comenzamos a investigar obteniendo una vista general de los parámetros en nuestros datos:

In [11]:
df_UK.describe()

,quantity,price,customerid
count,824364.000000,824364.000000,824364.000000
mean,12.414574,3.676800,15324.638504
std,188.976099,70.241388,1697.464450
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13975.000000
50%,5.000000,1.950000,15255.000000
75%,12.000000,3.750000,16797.000000
max,80995.000000,38970.000000,18287.000000


Notamos que existen valores negativos para la cantidad de producto y precio cero para algunas transaciones.

In [12]:
df_UK[(df_UK["quantity"]<0 )] #Registros con cantidades negativas

,invoice_no,stock_code,description,quantity,invoice_date,price,customerid,country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,12/1/09 10:33,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,12/1/09 10:33,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,12/1/09 10:33,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,12/1/09 10:33,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,12/1/09 10:33,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/11 9:57,0.83,14397.0,United Kingdom
1067002,C581499,M,Manual,-1,12/9/11 10:28,224.69,15498.0,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/11 11:57,10.95,15311.0,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/11 11:58,1.25,17315.0,United Kingdom


Despues de observar los registros que contienen cantidades negativas nos damos cuenta que corresponden a transaciones de cancelacion, por lo tanto son valores viables y no corresponden a errores en el dataset.

In [13]:
df_UK[(df_UK["price"]==0 )] #Registros con precio igual a cero 

,invoice_no,stock_code,description,quantity,invoice_date,price,customerid,country
4674,489825,22076,6 RIBBONS EMPIRE,12,12/2/09 13:34,0.0,16126.0,United Kingdom
6781,489998,48185,DOOR MAT FAIRY CAKE,2,12/3/09 11:19,0.0,15658.0,United Kingdom
16107,490727,M,Manual,1,12/7/09 16:38,0.0,17231.0,United Kingdom
18738,490961,22065,CHRISTMAS PUDDING TRINKET POT,1,12/8/09 15:25,0.0,14108.0,United Kingdom
18739,490961,22142,CHRISTMAS CRAFT WHITE FAIRY,12,12/8/09 15:25,0.0,14108.0,United Kingdom
...,...,...,...,...,...,...,...,...
1004540,577129,22464,HANGING METAL HEART LANTERN,4,11/17/11 19:52,0.0,15602.0,United Kingdom
1005014,577168,M,Manual,1,11/18/11 10:42,0.0,12603.0,Germany
1006110,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,11/18/11 13:23,0.0,12444.0,Norway
1011446,577696,M,Manual,1,11/21/11 11:57,0.0,16406.0,United Kingdom


Tenemos 71 registros de transaciones con precio igual a cero, ....

In [14]:
#df_UK[(df_UK["customerid"]==16321.0 )] #mirar para un cliente

### Descripciones de producto diferentes para un mismo codigo 

Los codigos de productos son asignados de forma unica, lo que significa que deberian existir igual numero de descripciones y codigos, pero en este dataset se encuentran descripciones con algunas variaciones:

In [15]:
print("Existen ",len(list(df_UK["stock_code"].unique()))," códigos de producto (stock_code) y ",len(list(df_UK["description"].unique())) ,"descripciones de producto (description)")

Existen  4646  códigos de producto (stock_code) y  5299 descripciones de producto (description)


observemos las diferencias en la descripcion para un mismo ```stock_code```: 

In [16]:
listaCodigos=list(df_UK["stock_code"].unique()) #lista de codigos no repetida
for codigo in listaCodigos: #recorremos la lista
    aux4=df_UK[(df_UK["stock_code"]==codigo)].groupby('description').size() #Obtenemos las diferenctes descripciones para un mismo codigo
    if aux4.shape[0]>1: #Si existe mas de una descripcion asociada al codigo
        print("-----------------------------------------------")
        for i in range(aux4.shape[0]): #recorra segun la cantidad de descripciones asociadas al codigo
            print(list(aux4.index)[i],aux4[i],sep="\t\t") #Imprima la descripcion y la cantidad de veces que aparece      

-----------------------------------------------
 WHITE CHERRY LIGHTS		221
WHITE CHERRY LIGHTS		217
-----------------------------------------------
STRAWBERRY CERAMIC TRINKET BOX		2106
STRAWBERRY CERAMIC TRINKET POT		84
-----------------------------------------------
DOOR MAT FANCY FONT HOME SWEET HOME		194
DOORMAT FANCY FONT HOME SWEET HOME		597
FANCY FONT HOME SWEET HOME DOORMAT		128
-----------------------------------------------
CAT BOWL 		99
ILLUSTRATED CAT BOWL 		161
-----------------------------------------------
DOG BOWL , CHASING BALL DESIGN		123
DOG BOWL CHASING BALL DESIGN		212
-----------------------------------------------
HEART MEASURING SPOONS LARGE		5
LARGE HEART MEASURING SPOONS		554
-----------------------------------------------
LUNCH BOX WITH CUTLERY FAIRY CAKES 		224
LUNCHBOX WITH CUTLERY FAIRY CAKES 		163
-----------------------------------------------
DOOR MAT BLACK FLOCK 		173
DOORMAT BLACK FLOCK 		304
-----------------------------------------------
 PEACE WOODEN

-----------------------------------------------
DOOR MAT NEW ENGLAND		427
DOORMAT NEW ENGLAND		757
-----------------------------------------------
RIBBON REEL POLKADOTS 		211
RIBBON REEL SPOTS DESIGN 		253
-----------------------------------------------
N0 SINGING METAL SIGN		35
NO SINGING METAL SIGN		1005
-----------------------------------------------
RETRO SPOT GIANT  TUBE MATCHES		110
RETROSPOT GIANT TUBE MATCHES		199
-----------------------------------------------
GLITTER CHRISTMAS HEART 		160
GLITTER HEART DECORATION		142
-----------------------------------------------
DOOR MAT 3 SMILEY CATS		252
DOORMAT 3 SMILEY CATS		341
-----------------------------------------------
DOOR MAT RED SPOT		373
DOORMAT RED RETROSPOT		802
DOORMAT RED SPOT		187
RED SPOTTY COIR DOORMAT		166
-----------------------------------------------
DOOR MAT HEARTS		289
DOORMAT HEARTS		883
-----------------------------------------------
SKULLS AND CROSSBONES WRAP		270
SKULLS WRAP		108
----------------------------

-----------------------------------------------
ENGLISH ROSE PLACEMATS		25
SET OF 4 ENGLISH ROSE PLACEMATS		252
-----------------------------------------------
PACK OF 12 PINK POLKADOT TISSUES		228
PACK OF 12 PINK SPOT TISSUES 		162
-----------------------------------------------
 BROWN  PIRATE TREASURE CHEST 		7
BROWN  PIRATE TREASURE CHEST 		86
-----------------------------------------------
BLUE FAIRY CAKE CHILD'S APRON		142
BLUE FAIRY CAKE CHILDRENS APRON		4
-----------------------------------------------
SET 3 RED SPOT TIN TEA,COFFEE,SUGAR		250
SET 3 RETROSPOT TEA,COFFEE,SUGAR		313
-----------------------------------------------
PINK  POLKADOT PLATE 		179
PINK  SPOTTY PLATE 		208
PINK POLKADOT PLATE 		18
-----------------------------------------------
RED RETROSPOT PEG BAG		465
RED SPOTTY PEG BAG		307
-----------------------------------------------
BLUE POLKADOT WRAP		272
BLUE SPOTS  WRAP		196
-----------------------------------------------
RED RETROSPOT BIG BOWL		53
RED SPOTTY BI

-----------------------------------------------
CERAMIC STRAWBERRY DESIGN MUG		238
CERAMIC STRWBERRY DESIGN MUG		201
-----------------------------------------------
RETRO SPOT CANDLE  MEDIUM		23
RETROSPOT CANDLE  MEDIUM		53
-----------------------------------------------
SET/10 IVORY POLKADOT PARTY CANDLES		125
SET/10 IVORY SPOTTY PARTY CANDLES		77
-----------------------------------------------
WALL MIRROR , RECT DIAMANTE, PINK/		26
WALL MIRROR RECTANGLE DIAMANTE PINK		23
-----------------------------------------------
SQUARE FLOOR CUSHION VINTAGE PINK 		3
SQUARE FLOOR CUSHION VINTAGE RED		51
-----------------------------------------------
 RIDGED GLASS T-LIGHT HOLDER		13
RIDGED GLASS T-LIGHT HOLDER		59
-----------------------------------------------
 HOME SWEET HOME  BLACKBOARD		35
HOME SWEET HOME  BLACKBOARD		4
-----------------------------------------------
IVORY KNITTED MUG COSY 		91
KNITTED MUG COSY IVORY COLOUR 		31
-----------------------------------------------
S/4 ROSE    BOT

-----------------------------------------------
DOOR MAT UNION FLAG		646
DOORMAT UNION FLAG		912
-----------------------------------------------
ROUND PURPLE CLOCK W/SUCKER		10
ROUND PURPLE CLOCK WITH SUCKER		27
-----------------------------------------------
PINK BUTTERFLY C/COVER 		3
PINK BUTTERFLY CUSHION COVER 		17
-----------------------------------------------
PINK YELLOW PATCH CUSHION COVER		14
PINK/YELLOW PATCH CUSHION COVER		5
-----------------------------------------------
BEADED CRYSTAL HEART PINK ON STICK		24
BEADED CRYSTAL HEART PINK\ON STICK		9
-----------------------------------------------
SMALL POP BOX FUNKY MONKEY		35
SMALL POP BOX,FUNKY MONKEY		48
-----------------------------------------------
DOOR MAT KISS MY.......		8
DOORMAT KISS MY.......		2
-----------------------------------------------
SMOKEY GREY COLOUR D.O.F. GLASS		42
SMOKEY GREY COLOUR GLASS		12
-----------------------------------------------
SILVER M.O.P. ORBIT NECKLACE		3
SILVER/MOP ORBIT NECKLACE		19
-

-----------------------------------------------
MONEY BOX FIRST ADE DESIGN		57
MONEY BOX FIRST AID DESIGN		10
-----------------------------------------------
MONEY BOX HOUSEKEEPING DESIGN		171
TIN CAN HOUSEKEEPING DESIGN		2
-----------------------------------------------
MAGIC DRAWING SLATE DOLLY GIRL 		393
MAGIC SLATE DOLLY GIRL 		4
-----------------------------------------------
MAGIC DRAWING SLATE SPACEBOY 		497
MAGIC SLATE SPACEBOY 		6
-----------------------------------------------
MAGIC DRAWING SLATE FIDO		8
MAGIC DRAWING SLATE PURDEY		341
MAGIC SLATE FIDO		6
-----------------------------------------------
MAGIC DRAWING SLATE BAKE A CAKE 		215
MAGIC SLATE BAKE A CAKE 		1
-----------------------------------------------
MAGIC DRAWING SLATE GO TO THE FAIR 		143
MAGIC SLATE GO TO THE FAIR 		2
-----------------------------------------------
MAGIC DRAWING SLATE LEAP FROG 		164
MAGIC SLATE LEAP FROG 		2
-----------------------------------------------
MAGIC DRAWING SLATE DINOSAUR		301
MA

-----------------------------------------------
ADULT APRON APPLE DELIGHT		41
APRON APPLE DELIGHT		116
-----------------------------------------------
POLKA DOT PEN		24
POLKADOT PEN		186
POLKADOT PENS		1
-----------------------------------------------
HOT WATER BOTTLE BABUSHKA 		423
HOT WATER BOTTLE BABUSHKA LARGE		14
-----------------------------------------------
CAKE TIN, ROUND, VINTAGE RED,CREAM		10
ROUND CAKE TIN VINTAGE RED		403
-----------------------------------------------
CAKE TIN, ROUND, VINTAGE MINT,IVORY		4
ROUND CAKE TIN VINTAGE GREEN		306
-----------------------------------------------
BISCUIT TIN VINTAGE RED		236
BISCUIT TIN, RED,IVORY, VINTAGE		7
-----------------------------------------------
BREAD BIN DINER STYLE IVORY		482
BREAD BIN, DINER STYLE, IVORY		11
-----------------------------------------------
BREAD BIN DINER STYLE MINT		164
BREAD BIN, DINER STYLE, MINT		5
-----------------------------------------------
3 TIER CAKE TIN RED AND CREAM		250
CAKE TIN, 3 TIER R

-----------------------------------------------
VINTAGE EMBOSSED HEART		88
VINTAGE ENGRAVED HEART		4
-----------------------------------------------
CRYSTAL CHANDELIER T-LIGHT HOLDER		1
FLOWERS CHANDELIER T-LIGHT HOLDER		46
-----------------------------------------------
MARIE ANTOIENETT TRINKET BOX GOLD		1
MARIE ANTOINETTE TRINKET BOX GOLD		25
-----------------------------------------------
MINIATURE ANTIQUE ROSE HOOK IVORY		247
MINITURE ANTIQUE ROSE HOOK IVORY		4
-----------------------------------------------
ZINC T-LIGHT HOLDER STAR LARGE		151
ZINC T-LIGHT HOLDER STARS LARGE		2
-----------------------------------------------
16 PC CUTLERY SET PANTRY DESIGN		1
16 PIECE CUTLERY SET PANTRY DESIGN		48
-----------------------------------------------
MISELTOE HEART WREATH CREAM		1
MISELTOE HEART WREATH WHITE		4
MISTLETOE HEART WREATH CREAM		107
-----------------------------------------------
DOORMAT VINTAGE LEAF		123
DOORMAT VINTAGE LEAVES DESIGN 		88
------------------------------------

-----------------------------------------------
DECORATIVE VINTAGE COFFEE  BOX		17
DECROTIVEVINTAGE COFFEE GRINDER BOX		4
VINTAGE COFFEE GRINDER BOX		25
-----------------------------------------------
 I LOVE LONDON MINI BACKPACK		70
 I LOVE LONDON MINI RUCKSACK		1
-----------------------------------------------
DOLLY GIRL MINI BACKPACK		152
DOLLY GIRL MINI RUCKSACK		2
-----------------------------------------------
SPACEBOY MINI BACKPACK		209
SPACEBOY MINI RUCKSACK		4
-----------------------------------------------
WOODLAND MINI BACKPACK		136
WOODLAND MINI RUCKSACK 		2
-----------------------------------------------
SET 12 COLOUR PENCILS DOILEY		3
SET 12 COLOURING PENCILS DOILEY		1
SET 12 COLOURING PENCILS DOILY		141
-----------------------------------------------
PLAYING CARDS VINTAGE DOILEY 		2
PLAYING CARDS VINTAGE DOILY 		86
-----------------------------------------------
SET 36 COLOUR PENCILS DOILEY		3
SET 36 COLOURING PENCILS DOILEY		1
SET 36 COLOURING PENCILS DOILY		110
-------

AQUI VAMOS 

In [17]:
a=list(df_UK["description"].unique())
len(a)

5299

In [18]:
df_UK["description"]=df_UK["description"].str.lstrip() #quitar espacio a la izquierda

In [19]:
df_UK["description"]=df_UK["description"].str.rstrip() #quitar espacio a la derecha

In [20]:
df_UK["description"]=df_UK["description"].str.replace(" ,",",") #quitar espacio a la derecha

In [21]:
df_UK["description"]=df_UK["description"].str.replace("  "," ") #quitar espacio a la derecha

In [22]:
df_UK["description"]=df_UK["description"].str.replace("CHARLIE + LOLA","CHARLIE+LOLA") #quitar espacio a la derecha

In [23]:
df_UK["description"]=df_UK["description"].str.replace("CHARLIE & LOLA","CHARLIE+LOLA") #quitar espacio a la derecha

In [24]:
df_UK["description"]=df_UK["description"].str.replace("CHARLIE LOLA","CHARLIE+LOLA") #quitar espacio a la derecha

In [25]:
df_UK["description"]=df_UK["description"].str.replace("CHARLIE AND LOLA","CHARLIE+LOLA") #quitar espacio a la derecha

In [26]:
df_UK["description"]=df_UK["description"].str.replace(" + ","+") #quitar espacio a la derecha

In [27]:
b=list(df_UK["description"].unique())
len(b)

5246

In [28]:
b.sort()

In [29]:
b[500:550]

['BLOSSOM IMAGES GIFT WRAP SET',
 'BLOSSOM IMAGES NOTEBOOK SET',
 'BLOSSOM IMAGES SCRAP BOOK SET',
 'BLUE & WHITE BREAKFAST TRAY',
 'BLUE 3 PIECE MINI DOTS CUTLERY SET',
 'BLUE 3 PIECE POLKADOT CUTLERY SET',
 'BLUE 5 PATCH FLOWER CUSHION COVER',
 "BLUE BABY BOY'S MEMORY BOX",
 'BLUE BAROQUE FLOCK CANDLE HOLDER',
 'BLUE BEADS+HAND PHONE CHARM',
 'BLUE BIRDHOUSE DECORATION',
 'BLUE BLOSSOM HAIR CLIP',
 'BLUE BREAKFAST CUP AND SAUCER',
 'BLUE BUNNY EASTER EGG BASKET',
 'BLUE BUTTERFLY ON SPRING',
 'BLUE CALCULATOR RULER',
 'BLUE CAT BISCUIT BARREL PINK HEART',
 'BLUE CHALET BIRDFEEDER',
 'BLUE CHARLIE+LOLA PERSONAL DOORSIGN',
 'BLUE CHECK BAG W HANDLE 34X20CM',
 'BLUE CHENILLE SHAGGY CUSHION COVER',
 'BLUE CHERRY BLOSSOM CUP & SAUCER',
 'BLUE CIRCLES DESIGN MONKEY DOLL',
 'BLUE CIRCLES DESIGN TEDDY',
 'BLUE CLIMBING HYDRANGA ART FLOWER',
 'BLUE COAT RACK PARIS FASHION',
 'BLUE COL METAL FOLDING CHAIR',
 'BLUE CRUSOE CHECK LAMPSHADE',
 'BLUE CRYSTAL BOOT PHONE CHARM',
 'BLUE CUSHION COVER 

In [30]:
for i in b:
 if i.find(",")>0:
    print(i)

ACRYLIC BEAD CHAIN, BLUE
ACRYLIC BEAD CHAIN, PINK
ACRYLIC BEAD GARLAND,PINK
ACRYLIC HANGING JEWEL,BLUE
ACRYLIC HANGING JEWEL,PINK
ACRYLIC JEWEL ANGEL,BLUE
ACRYLIC JEWEL ANGEL,PINK
ACRYLIC JEWEL ICICLE, BLUE
ACRYLIC JEWEL ICICLE, PINK
ACRYLIC JEWEL SNOWFLAKE, PINK
ACRYLIC JEWEL SNOWFLAKE,BLUE
ACRYLIC JEWEL SNOWFLAKE,PINK
AIRLINE LOUNGE,METAL SIGN
ART LIGHTS,FUNK MONKEY
ASS RETROSPOOF MUGS,SPORTS
BATHROOM SCALES, FAG BREAK
BATHROOM SCALES, TROPICAL BEACH
BIRTHDAY CARD, RETRO SPOT
BISCUIT TIN, MINT,IVORY, VINTAGE
BISCUIT TIN, RED,IVORY, VINTAGE
BLACK TEA,COFFEE,SUGAR JARS
BREAD BIN, DINER STYLE, IVORY
BREAD BIN, DINER STYLE, MINT
BREAD BIN, DINER STYLE, PINK
BREAD BIN, DINER STYLE, RED
BROCADE PURSE,SMALL ARCH BUTTON
BUNTING, SPOTTY
CAKE TIN, 3 TIER MINT/IVORY
CAKE TIN, 3 TIER RED/IVORY
CAKE TIN, ROUND, VINTAGE MINT,IVORY
CAKE TIN, ROUND, VINTAGE RED,CREAM
CAKESTAND, 3 TIER, LOVEHEART
CANDLE IN JAR COLLAGE, VANILLA
CARD, BILLBOARD FONT
CARD, GINGHAM ROSE
CAT BOWL, ENAMEL, CREAM COLOUR
CAT

In [31]:
df_UK[['main description']] = df_UK['description'].str.split(',', expand=True)[0]

In [32]:
#df_UK[['complement description']] = df_UK['description'].str.split(',', expand=True)[1:]

In [33]:
#df_UK[['main description', 'complement description']] = df_UK['description'].str.split(',', maxsplit=1)


In [34]:
df_UK[(df_UK['description'].str.find(",")>0)].head(10)

,invoice_no,stock_code,description,quantity,invoice_date,price,customerid,country,main description
9,489435,22349,"DOG BOWL, CHASING BALL DESIGN",12,12/1/09 7:46,3.75,13085.0,United Kingdom,DOG BOWL
91,489440,22349,"DOG BOWL, CHASING BALL DESIGN",8,12/1/09 9:43,3.75,18087.0,United Kingdom,DOG BOWL
107,489442,84251J,"GREETING CARD, THE BLACK CAT",12,12/1/09 9:46,0.42,13635.0,United Kingdom,GREETING CARD
144,489445,22241,"GARLAND, WOODEN "" HAPPY EASTER""",24,12/1/09 9:57,1.25,17519.0,United Kingdom,GARLAND
162,489446,22356,"CHARLOTTE BAG, PINK/WHITE SPOTS",10,12/1/09 10:06,0.85,13758.0,United Kingdom,CHARLOTTE BAG
264,489465,21707,"FOLDING UMBRELLA, BLACK/BLUE SPOT",12,12/1/09 10:52,4.95,13767.0,United Kingdom,FOLDING UMBRELLA
267,489465,21706,"FOLDING UMBRELLA, RED/WHITE SPOT",12,12/1/09 10:52,4.95,13767.0,United Kingdom,FOLDING UMBRELLA
342,489514,82567,"AIRLINE LOUNGE,METAL SIGN",1,12/1/09 11:21,2.10,15311.0,United Kingdom,AIRLINE LOUNGE
395,489517,85129A,"WHITE PEARL BEADED HEART, SMALL",1,12/1/09 11:34,1.25,16329.0,United Kingdom,WHITE PEARL BEADED HEART
550,489522,22197,"POPCORN HOLDER, SMALL",10,12/1/09 11:45,0.85,15998.0,United Kingdom,POPCORN HOLDER


## DATASET: retailer_pricing 

Las columnas presentadas en este archivo son:

|CAMPO   |TIPO   |DESCRIPCION   |
|---|---|---|
|store|STRING|El minorista bajo el cual se incluye este producto*|
|title| STRING| El título del producto.|
|sku |STRING |La unidad de existencias específica del minorista (SKU).|
|groupid |STRING|El SKU a nivel de grupo, identifica la lista de productos iguales o similares en diferentes minoristas|
|brand |STRING |La marca.|
|color |STRING |El color.|
|sizing |STRING |La talla.|
|category |STRING |La categoria.|
|subcategory |STRING |La Subcategoria.|
|price |FLOAT |El precio del producto, en USD.|
|date |STRING |La marca de tiempo para cuando se registraron los detalles (formato:yyyy-mm-dd).|


*minoristas: (0=amazon.com, 1=zappos.com, 2=macys.com, 3=neimanmarcus.com, 4=saksfifthavenue.com, 5=bloomingdales.com)

In [35]:
#df_rp=df_retailer_pricing=pd.read_csv(ruta+"/data/retailer_pricing/retailer_pricing.csv")

In [36]:
#df_rp.shape

## DATASET: amazon_beauty_reviews 

Las columnas presentadas en este archivo son:

|CAMPO   |TIPO   |DESCRIPCION   |
|---|---|---|
|title |STRING |El nombre del producto|
brand |STRING |El nombre de la marca.
rank |FLOAT |El rango del producto en la categoría Belleza.
asin |STRING |El ID del producto
description  |LIST |La descripción del producto.
also_view |LIST |Productos relacionados que vieron otros clientes.
also_buy |LIST |Productos relacionados que compraron otros clientes.
price |FLOAT |El precio del producto en USD
similar_item |LIST |Tabla de productos similares.

In [37]:
#df_ab=df_amazon_beauty=pd.read_csv(ruta+"/data/amazon_reviews/beauty_metadata.csv") 

In [38]:
#df_ab.shape